## Importing Dependencies


In [34]:
from googleapiclient.discovery import build
import pandas as pd
import os
import webbrowser

## Creating Dataset


In [47]:
os.makedirs('data', exist_ok=True)
words_path = os.path.join('data', 'words.csv')

In [48]:
word_list = ["nude ass", "nude pussy", "ass twerk", "sex videos",
             "porn videos", "desi sex", "chut", "xxx", "try not to cum", "sexx"]

df = pd.DataFrame({'word': word_list})
df.to_csv(words_path, index=False)

In [49]:
def scrape_youtube_videos(query):
    api_key = os.environ.get("YOUTUBE_API_KEY")
    youtube = build('youtube', 'v3', developerKey=api_key)

    video_links = []
    video_ids = []
    next_page_token = None

    while True:
        search_response = youtube.search().list(
            q=query,
            type='video',
            part='id',
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in search_response['items']:
            video_links.append(
                'https://www.youtube.com/watch?v=' + item['id']['videoId'])
            video_ids.append(item['id']['videoId'])

        next_page_token = search_response.get('nextPageToken')

        if not next_page_token:
            break

    df = pd.DataFrame({'link': video_links, 'id': video_ids})
    p = os.path.join('data', query + '.csv')
    df.to_csv(p, index=False)

In [50]:
wd = pd.read_csv(words_path)
wd = wd['word'].tolist()

In [41]:
for word in wd:
    scrape_youtube_videos(word)

In [52]:
# add content of one file to another
for word in wd:
    p = os.path.join('data', word + '.csv')
    if os.path.exists(p):
        df = pd.read_csv(p)
        p2 = os.path.join('data', 'links.csv')
        df.to_csv(p2, mode='a', header=False, index=False)

In [55]:
# add column names to csv file
df = pd.read_csv('data/links.csv')
df.columns = ['index','link', 'id']
df.to_csv('data/links.csv', index=False)


In [66]:
df = pd.read_csv('data/links.csv')
df = df.drop('index', axis=1)
# df.head()

In [67]:
video_id = df['id']
video_id = video_id.sample(5)
# video_id

## Playing Video in Browser


In [64]:
def play_youtube_video(video_id):
    for id in video_id:
        url = f"https://www.youtube.com/watch?v={id}"
        webbrowser.open(url)

In [65]:
play_youtube_video(video_id)